# runtime

In [ ]:
open rust_operators
open sm'_operators

In [ ]:
//// test

open testing
open file_system_operators

## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::Arg\")>]\n#endif\ntype clap_Arg = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::ArgAction\")>]\n#endif\ntype clap_ArgAction = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::Command\")>]\n#endif\ntype clap_Command = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::ArgMatches\")>]\n#endif\ntype clap_ArgMatches = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::builder::ValueRange\")>]\n#endif\ntype clap_builder_ValueRange = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::builder::ValueParser\")>]\n#endif\ntype clap_builder_ValueParser = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::builder::PossibleValue\")>]\n#endif\ntype clap_builder_PossibleValue = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Child\")>]\n#endif\ntype std_process_Child = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::ChildStderr\")>]\n#endif\ntype std_process_ChildStderr = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::ChildStdout\")>]\n#endif\ntype std_process_ChildStdout = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::ChildStdin\")>]\n#endif\ntype std_process_ChildStdin = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Command\")>]\n#endif\ntype std_process_Command = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::ExitStatus\")>]\n#endif\ntype std_process_ExitStatus = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Output\")>]\n#endif\ntype std_process_Output = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Stdio\")>]\n#endif\ntype std_process_Stdio = class end"

In [ ]:
inl types () =
    types ()
    am'.types ()
    async.types ()
    env.types ()
    rust.types ()
    seq.types ()
    sm'.types ()
    threading.types ()

In [ ]:
//// test

inl types () =
    file_system.types ()
    types ()

## process_child

In [ ]:
nominal process_child = $'std_process_Child'

## process_child_stderr

In [ ]:
nominal process_child_stderr = $"std_process_ChildStderr"

inl process_child_stderr (child : rust.ref' (rust.mut' process_child)) : rust.ref' (rust.mut' (optionm'.option' process_child_stderr)) =
    !\\(child, $'"&mut $0.stderr"')

## process_child_stdout

In [ ]:
nominal process_child_stdout = $"std_process_ChildStdout"

inl process_child_stdout (child : rust.ref' (rust.mut' process_child)) : rust.ref' (rust.mut' (optionm'.option' process_child_stdout)) =
    !\\(child, $'"&mut $0.stdout"')

## process_child_stdin

In [ ]:
nominal process_child_stdin = $"std_process_ChildStdin"

inl process_child_stdin (child : rust.ref' (rust.mut' process_child)) : rust.ref' (rust.mut' (optionm'.option' process_child_stdin)) =
    !\\(child, $'"&mut $0.stdin"')

## process_command

In [ ]:
nominal process_command = $'std_process_Command'

## process_stdio

In [ ]:
nominal process_stdio = $'std_process_Stdio'

## process_output

In [ ]:
nominal process_output = $'std_process_Output'

## process_exit_status

In [ ]:
nominal process_exit_status = $'std_process_ExitStatus'

## process_output_status

In [ ]:
inl process_output_status (output : process_output) : process_exit_status =
    !\\(output, $'"$0.status"')

## process_exit_status_code

In [ ]:
inl process_exit_status_code (status : process_exit_status) : optionm'.option' i32 =
    !\\(status, $'"$0.code()"')

## os_platform

In [ ]:
nominal os_platform' = $'System.Runtime.InteropServices.OSPlatform'

union os_platform =
    | Windows

inl os_platform = function
    | Windows => $'`os_platform'.Windows' : os_platform'

## is_os_platform

In [ ]:
inl is_os_platform (x : os_platform') : bool =
    x |> $'System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform'

## is_windows

In [ ]:
inl is_windows () : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            Windows |> os_platform |> is_os_platform
        | Rust (Native) => fun () =>
            !\($'"cfg\!(windows)"')
        | _ => fun () => null ()

## get_executable_suffix

In [ ]:
inl get_executable_suffix () =
    if is_windows ()
    then ".exe"
    else ""

In [ ]:
//// test

get_executable_suffix ()

## quoted

In [ ]:
union quoted =
    | Quoted
    | NotQuoted

## split_command

In [ ]:
union command_parse_step =
    | Start
    | Path : quoted
    | Arguments

inl split_command (command : string) =
    let rec loop (path, args) (chars : list char) step =
        match step, chars with
        | _, (('"' | '\'') :: tail) when path = "" => loop (path, args) tail (Path Quoted)
        | Path (Quoted), (('"' | '\'') :: tail) => loop (path, args) tail (Path NotQuoted)
        | Path (Quoted), (' ' :: tail) => loop ($'$"{!path} "', args) tail (Path Quoted)
        | (Start | Path _), (' ' :: tail) => loop (path, args) tail Arguments
        | Arguments, (char :: tail) => loop (path, $'$"{!args}{!char}"') tail Arguments
        | _, (char :: tail) => loop ($'$"{!path}{!char}"', args) tail step
        | _ => path |> sm'.replace @"\" "/", args
    loop (join "", "") (command |> sm'.to_char_array |> am'.to_list' |> listm'.unbox) Start

In [ ]:
//// test

split_command ""
|> _assert_eq ("", "")

split_command "/a/b/c"
|> _assert_eq ("/a/b/c", "")

split_command "cat file.txt"
|> _assert_eq ("cat", "file.txt")

split_command $'"""..\\..\\file.exe file1.txt file2.txt"""'
|> _assert_eq ("../../file.exe", "file1.txt file2.txt")

split_command $'\@"c:\\dir\\file.exe ""file1.txt file2.txt"""'
|> _assert_eq ("c:/dir/file.exe", $'\@"""file1.txt file2.txt"""')

split_command $'\@"""..\\..\\dir name\\file.exe"" ""file 1.txt"" file2.txt"'
|> _assert_eq ("../../dir name/file.exe", $'\@"""file 1.txt"" file2.txt"')

split_command $'\@"""..\\..\\file 1.exe"" -c \\\\""echo 1\\\\"""'
|> _assert_eq ("../../file 1.exe", $'\@"-c \\\\""echo 1\\\\"""')

split_command $'\@"..\\..\\file 1.exe -c \\\\""echo 1\\\\"""'
|> _assert_eq ("../../file", $'\@"1.exe -c \\\\""echo 1\\\\"""')

assert_eq / actual: struct ("", "") / expected: struct ("", "")
assert_eq / actual: struct ("/a/b/c", "") / expected: struct ("/a/b/c", "")
assert_eq / actual: struct ("cat", "file.txt") / expected: struct ("cat", "file.txt")
assert_eq / actual: struct ("../../file.exe", "file1.txt file2.txt") / expected: struct ("../../file.exe", "file1.txt file2.txt")
assert_eq / actual: struct ("c:/dir/file.exe", ""file1.txt file2.txt"") / expected: struct ("c:/dir/file.exe", ""file1.txt file2.txt"")
assert_eq / actual: struct ("../../dir name/file.exe", ""file 1.txt" file2.txt") / expected: struct ("../../dir name/file.exe", ""file 1.txt" file2.txt")
assert_eq / actual: struct ("../../file 1.exe", "-c \\"echo 1\\"") / expected: struct ("../../file 1.exe", "-c \\"echo 1\\"")
assert_eq / actual: struct ("../../file", "1.exe -c \\"echo 1\\"") / expected: struct ("../../file", "1.exe -c \\"echo 1\\"")


## process

In [ ]:
nominal process = $'System.Diagnostics.Process'
nominal process_start_info = $'System.Diagnostics.ProcessStartInfo'
nominal data_received_event_args = $'System.Diagnostics.DataReceivedEventArgs'

inl new_process (process_start_info : process_start_info) : process =
    $'new `process (StartInfo = !process_start_info)'

inl process_start (process : process) : bool =
    $'!process.Start' ()

inl process_exit_code (process : process) : i32 =
    $'!process.ExitCode'

inl process_id (process : process) : i32 =
    $'!process.Id'

inl process_has_exited (process : process) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!process.HasExited'
        | _ => fun () => null ()

inl process_kill (process : process) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!process.Kill' ()
        | _ => fun () => null ()

inl process_begin_error_read_line (process : process) : () =
    $'!process.BeginErrorReadLine' ()

inl process_begin_output_read_line (process : process) : () =
    $'!process.BeginOutputReadLine' ()

inl process_add_output_data_received fn (process : process) : () =
    $'!process.OutputDataReceived.Add !fn '

inl process_add_error_data_received fn (process : process) : () =
    $'!process.ErrorDataReceived.Add !fn '

inl process_wait_for_exit_async (ct : threading.cancellation_token) (process : process) : async.task () =
    $'!process.WaitForExitAsync !ct '

inl event_data (e : data_received_event_args) : string =
    $'!e.Data'

## execution_line

In [ ]:
type execution_line =
    {
        process_id : int
        line : string
        error : bool
    }

## execution_options

In [ ]:
type execution_options =
    {
        command : string
        cancellation_token : optionm'.option' threading.cancellation_token
        environment_variables : array_base (string * string)
        on_line : optionm'.option' (execution_line -> async.async ())
        stdin : optionm'.option' (threading.arc (threading.mutex process_child_stdin) -> ())
        trace : bool
        working_directory : optionm'.option' string
    }

inl execution_options (fn : execution_options -> execution_options) : execution_options =
    {
        command = ""
        cancellation_token = None |> optionm'.box
        environment_variables = ;[]
        on_line = None |> optionm'.box
        stdin = None |> optionm'.box
        trace = true
        working_directory = None |> optionm'.box
    }
    |> fn

## execute_with_options_async

In [ ]:
let execute_with_options_async (options : execution_options) : _ (i32 * string) =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                inl file_name, arguments = options.command |> split_command
                inl working_directory = options.working_directory |> optionm'.unbox |> optionm'.default_value ""

                trace Debug (fun () => $'$"execute_with_options_async / options: %A{!options}"') (join _locals)

                inl utf8 = sm'.encoding_utf8 ()
                inl start_info : process_start_info = $'System.Diagnostics.ProcessStartInfo (Arguments = !arguments, StandardOutputEncoding = !utf8, WorkingDirectory = !working_directory, FileName = !file_name, CreateNoWindow = true, RedirectStandardError = true, RedirectStandardOutput = true, UseShellExecute = false)'

                (a options.environment_variables : _ i32 _)
                |> am.iter fun key, value =>
                    $'!start_info.EnvironmentVariables.[!key] <- !value '

                inl proc = start_info |> new_process |> use
                inl output : _ string = threading.new_concurrent_stack ()

                inl event error (e : data_received_event_args) = async.new_async fun () =>
                    inl data = e |> event_data
                    if data <> null () then
                        match options.on_line |> optionm'.unbox with
                        | Some on_line =>
                            on_line
                                {
                                    process_id = proc |> process_id
                                    line = data
                                    error = error
                                }
                            |> async.do
                        | None => ()

                        inl text =
                            if error
                            then $'$"\! {!data}"'
                            else $'$"> {!data}"'
                        if options.trace
                        then trace Verbose (fun () => text) _locals
                        else text |> console.write_line

                        inl l = if error then $'"\\u001b[7;4m"' else ""
                        inl r = if error then $'"\\u001b[0m"' else ""
                        output |> threading.concurrent_stack_push $'$"{!l}{!data}{!r}"'

                proc |> process_add_output_data_received (event false >> async.start_immediate)
                proc |> process_add_error_data_received (event true >> async.start_immediate)

                if proc |> process_start |> not
                then failwith $'$"execute_with_options_async / process_start error"'

                proc |> process_begin_error_read_line
                proc |> process_begin_output_read_line

                inl ct =
                    options.cancellation_token
                    |> optionm'.unbox
                    |> optionm'.default_with threading.token_none
                    |> async.merge_cancellation_token_with_default_async
                    |> async.let'

                ct |> threading.token_register fun () =>
                    if proc |> process_has_exited |> not
                    then proc |> process_kill
                |> use
                |> ignore

                inl exit_code : i32 =
                    fun () =>
                        try_unit
                            fun () =>
                                proc
                                |> process_wait_for_exit_async ct
                                |> async.await_task
                                |> async.do
                                proc |> process_exit_code |> return
                            fun ex =>
                                // with :? System.Threading.Tasks.TaskCanceledException as ex =>
                                inl ex' = ex |> sm'.format_exception
                                output |> threading.concurrent_stack_push ex'
                                inl ex : async.task_canceled_exception = ex |> unbox
                                trace Warning (fun () => $'$"execute_with_options_async / WaitForExitAsync / ex: %A{!ex}"') (join _locals)
                                (limit.min : i32) |> return
                    |> async.new_async_unit
                    |> async.let'

                inl output =
                    output
                    |> seq.rev''
                    |> fun x => x : seq.seq' string
                    |> sm'.concat "\n"

                trace Debug (fun () =>
                    $'$"execute_with_options_async / exit_code: {!exit_code} / output.Length: {!output.Length}"'
                ) (join _locals)

                (exit_code, output) |> return
            |> async.new_async_unit
        | _ => fun () =>
            global "#if FABLE_COMPILER\n[<CompilationRepresentation (CompilationRepresentationFlags.ModuleSuffix)>]\nmodule System =\n module Diagnostics =\n  type Process = unit\n  type DataReceivedEventArgs = unit\n#endif"
            null ()

## execute_async

In [ ]:
inl execute_async command =
    execution_options fun x => { x with
        command = command
    }
    |> execute_with_options_async

In [ ]:
//// test

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
inl file_name = "test.txt"
inl path = temp_folder </> file_name

inl content = "╭─[ 你好，世界！こんにちは世界！ ]─╮"
fun () =>
    inl exit_code, result = execute_async $'\@$"pwsh -c ""Get-Content {!path}"""' |> async.let'
    exit_code |> join _assert_eq 1
    result |> _assert_string_contains "not exist"

    content |> file_system.write_all_text_async path |> async.do

    execution_options fun x => { x with
        command = $'\@$"cat ""{!file_name}"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options_async
    |> async.let'
    |> ignore

    execution_options fun x => { x with
        command = $'\@$"pwsh -c ""[System.Console]::OutputEncoding = [System.Text.Encoding]::UTF8; Get-Content {!file_name}"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options_async
    |> async.return_await
|> async.new_async_unit
|> async.run_with_timeout 10000
|> function
    | Some (exit_code, output) =>
        exit_code |> join _assert_eq 0i32
        output |> join _assert_eq content
        true
    | _ => false
|> _assert_eq true

00:00:00   debug #1 execute_with_options_async / options: struct (None,
        "pwsh -c "Get-Content /tmp/!dotnet-repl/20240520-1342-3365-6577-60000003f314/test.txt"",
        [||], None, None, true, None)
00:00:00 verbose #2 ! Get-Content: Cannot find path '/tmp/!dotnet-repl/20240520-1342-3365-6577-60000003f314/test.txt' because it does not exist.
00:00:00   debug #3 execute_with_options_async / exit_code: 1 / output.Length: 150
assert_eq / actual: 1 / expected: 1
assert_string_contains / actual: "not exist" / expected: "Get-Content: Cannot find path '/tmp/!dotnet-repl/20240520-1342-3365-6577-60000003f314/test.txt' because it does not exist."
00:00:00   debug #4 execute_with_options_async / options: struct (None, "cat "test.txt"", [||], None, None, true,
        Some "/tmp/!dotnet-repl/20240520-1342-3365-6577-60000003f314")
00:00:00 verbose #5 > ╭─[ 你好，世界！こんにちは世界！ ]─╮
00:00:00   debug #6 execute_with_options_async / exit_code: 0 / output.Length: 22
00:00:00   debug #7 execute_with_op

In [ ]:
//// test

file_system.types ()
inl temp_dir, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
fun () =>
    inl path = temp_dir </> "test.txt"
    "0" |> file_system.write_all_text_async path |> async.do

    inl cts = threading.new_cancellation_token_source ()
    trace Debug (fun () => "1") _locals
    inl result =
        execution_options fun x => { x with
            command = $'\@$"pwsh -c ""Get-Content {!path}"""'
            cancellation_token = cts |> threading.cancellation_source_token |> Some |> optionm'.box
        }
        |> execute_with_options_async
        |> async.start_child
        |> async.let'
    trace Debug (fun () => "2") _locals
    async.sleep 100 |> async.do
    trace Debug (fun () => "3") _locals
    cts |> threading.cancellation_source_cancel
    trace Debug (fun () => "4") _locals
    inl exit_code, output = result |> async.let'
    trace Debug (fun () => "5") _locals
    (exit_code, output) |> return
|> async.new_async_unit
|> async.run_with_timeout 10000
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq -2147483648i32
        output |> _assert_eq (join "System.Threading.Tasks.TaskCanceledException: A task was canceled.")
        true
    | _ => false
|> _assert_eq true

00:00:00   debug #1 1
00:00:00   debug #2 2
00:00:00   debug #3 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "pwsh -c "Get-Content /tmp/!dotnet-repl/20240520-1342-3571-7115-70000032b7d9/test.txt"",
        [||], None, None, true, None)
00:00:00   debug #4 3
00:00:00   debug #5 4
00:00:00 warning #6 execute_with_options_async / WaitForExitAsync / ex: System.Threading.Tasks.TaskCanceledException: A task was canceled.
00:00:00   debug #7 execute_with_options_async / exit_code: -2147483648 / output.Length: 66
00:00:00   debug #8 5
assert_eq / actual: -2147483648 / expected: -2147483648
assert_eq / actual: "System.Threading.Tasks.TaskCanceledException: A task was canceled." / expected: "System.Threading.Tasks.TaskCanceledException: A task was canceled."
assert_eq / actual: true / expected: true


## split_args

In [ ]:
union args_parse_step =
    | Start
    | Quoted : char
    | Escaped : u8 * char

let split_args (args : string) : array_base string =
    let rec loop (acc, current) (chars : list char) (step, last) =
        // trace Verbose (fun () => $'$"split_args / current: %A{!current} / acc: %A{!acc} / step: %A{!step} / last: %A{!last}"') _locals
        match step, last, chars with
        | Start, _, ('"' :: tail) =>
            loop (acc, "") tail (Quoted '"', step)
        | Escaped _, _, (('\\' | '\`') & c :: tail) =>
            loop (acc, current) tail (Quoted c, step)
        | Escaped _, Start, ('"' :: tail) =>
            loop (acc, current) tail (step, step)
        | Quoted ('"'), Quoted _, ('"' :: tail) =>
            loop (acc, current) tail (Start, step)
        | Escaped (0, ('\\' | '\`') & c), Quoted ('"'), (c' :: tail) =>
            loop (acc, $'$"{!current}{!c}{!c'}"') tail (Quoted '"', step)
        | Quoted (('\\' | '\`') as c), _, ('"' :: tail)
                when last <>. Escaped (0, '\\') && last <>. Escaped (0, '\`') =>
            loop (acc, $'$"{!current}{!c}\\\""') tail (Quoted '"', step)
        | Quoted (('\\' | '\`') as c), Quoted _, ('"' :: tail) =>
            loop (acc, $'$"{!current}{!c}\\\""') tail (Quoted '"', step)
        | Quoted ('\\' | '\`'), _, ('"' :: tail) =>
            loop (acc, current) tail (Start, step)
        | Escaped (n, c), _, ('"' :: tail) =>
            loop (acc, $'$"{!current}{!c}\\\""') tail (Escaped (n, c), step)
        | Quoted _, Quoted _, (('\\' | '\`') & c :: tail) =>
            loop (acc, current) tail (Escaped (0, c), step)
        | Quoted _, _, (('\\' | '\`') & c :: tail) =>
            loop (acc, current) tail (Escaped (1, c), step)
        | Quoted _, Escaped (0, _), ('"' :: tail) =>
            loop (acc, $'$"{!current}"') tail (Quoted '"', step)
        | Quoted _, _, ('"' :: tail) =>
            loop (acc ++ [ current ], "") tail (Start, step)
        | (Escaped _ | Quoted _), _, (' ' :: tail) =>
            loop (acc, $'$"{!current} "') tail (step, step)
        | _, _, (('\\' | '\`') & c :: tail) =>
            loop (acc, current) tail (Escaped (0, c), step)
        | _, _, (' ' :: tail) =>
            loop ((if current = "" then acc else acc ++ [ current ]), "") tail (step, step)
        | Escaped (1, _), _, (char :: tail) when last <>. step =>
            loop (acc, $'$"{!current}\\{!char}"') tail (last, step)
        | _, _, (char :: tail) =>
            loop (acc, $'$"{!current}{!char}"') tail (step, step)
        | _ =>
            (if current = "" then acc else acc ++ [ current ]), current
    loop
        ([], "")
        (
            args
            |> optionm'.of_obj
            |> optionm'.unbox
            |> optionm'.default_value ""
            |> sm'.to_char_array
            |> am'.to_list'
            |> listm'.unbox
        )
        (Start, Start)
    |> fst
    |> listm'.box
    |> listm'.to_array'
    |> fun (a x : _ i32 _) => x

In [ ]:
//// test

"a b \"c d\" e"
|> split_args
|> _assert_eq' ;[ "a"; "b"; "c d"; "e" ]

assert_eq' / actual: [|"a"; "b"; "c d"; "e"|] / expected: [|"a"; "b"; "c d"; "e"|]


In [ ]:
//// test

"a b e"
|> split_args
|> _assert_eq' ;[ "a"; "b"; "e" ]

assert_eq' / actual: [|"a"; "b"; "e"|] / expected: [|"a"; "b"; "e"|]


In [ ]:
//// test

"\"a b\" \"e\" \"f\""
|> split_args
|> _assert_eq' ;[ "a b"; "e"; "f" ]

assert_eq' / actual: [|"a b"; "e"; "f"|] / expected: [|"a b"; "e"; "f"|]


In [ ]:
//// test

"a -b \"c \\\"d\\\"\""
|> split_args
|> _assert_eq' ;[ "a"; "-b"; "c \\\"d\\\"" ]

assert_eq' / actual: [|"a"; "-b"; "c \"d\""|] / expected: [|"a"; "-b"; "c \"d\""|]


In [ ]:
//// test

"a -b \"c \\\"d\\\" e\""
|> split_args
|> _assert_eq' ;[ "a"; "-b"; "c \\\"d\\\" e" ]

assert_eq' / actual: [|"a"; "-b"; "c \"d\" e"|] / expected: [|"a"; "-b"; "c \"d\" e"|]


In [ ]:
//// test

"a -b \"c \`\"d\`\" e\""
|> split_args
|> _assert_eq' ;[ "a"; "-b"; "c \`\"d\`\" e" ]

assert_eq' / actual: [|"a"; "-b"; "c `"d`" e"|] / expected: [|"a"; "-b"; "c `"d`" e"|]


In [ ]:
//// test

$'$"--text \\\\\\"\'\'\' value \'\'\'\\\\\\" "'
|> split_args
|> _assert_eq' ;[ "--text"; "''' value '''" ]

assert_eq' / actual: [|"--text"; "''' value '''"|] / expected: [|"--text"; "''' value '''"|]


In [ ]:
//// test

$'\@$"run ""get c:\\test.txt"""'
|> split_args
|> _assert_eq' ;[ "run"; "get c:\\test.txt" ]

assert_eq' / actual: [|"run"; "get c:\test.txt"|] / expected: [|"run"; "get c:\test.txt"|]


In [ ]:
//// test

"pwsh -c \"$x -replace '(id=`\"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }\""
|> split_args
|> _assert_eq' ;[ "pwsh"; "-c"; "$x -replace '(id=`\"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }" ]

assert_eq' / actual: [|"pwsh"; "-c";
  "$x -replace '(id=`"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }"|] / expected: [|"pwsh"; "-c";
  "$x -replace '(id=`"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }"|]


In [ ]:
//// test

$'$"pwsh -c \\\"$x -replace \'(id=\\\\\\"cell-id=)[a-fA-F0-9]{{8}}\', {{ \`$_[1] + \`$counter++ }}\\\""'
|> split_args
|> _assert_eq' ;[ "pwsh"; "-c"; "$x -replace '(id=\\\"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }" ]

assert_eq' / actual: [|"pwsh"; "-c";
  "$x -replace '(id=\"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }"|] / expected: [|"pwsh"; "-c";
  "$x -replace '(id=\"cell-id=)[a-fA-F0-9]{8}', { `$_[1] + `$counter++ }"|]


In [ ]:
//// test

$'$"a --b --c d --e \\\"c:/f f/test.hangul.md\\\" --output \\\"c:/md.pdf\\\" --set tex.template.add \\\"\\\\usepackage{{cjkutf8-ko}}\\\" rendering.highlight.theme \\\"Solarized (dark)\\\""'
|> split_args
|> _assert_eq' ;[ "a"; "--b"; "--c"; "d"; "--e"; "c:/f f/test.hangul.md"; "--output"; "c:/md.pdf"; "--set"; "tex.template.add"; "\\usepackage{cjkutf8-ko}"; "rendering.highlight.theme"; "Solarized (dark)" ]

assert_eq' / actual: [|"a"; "--b"; "--c"; "d"; "--e"; "c:/f f/test.hangul.md"; "--output";
  "c:/md.pdf"; "--set"; "tex.template.add"; "\usepackage{cjkutf8-ko}";
  "rendering.highlight.theme"; "Solarized (dark)"|] / expected: [|"a"; "--b"; "--c"; "d"; "--e"; "c:/f f/test.hangul.md"; "--output";
  "c:/md.pdf"; "--set"; "tex.template.add"; "\usepackage{cjkutf8-ko}";
  "rendering.highlight.theme"; "Solarized (dark)"|]


## stdin_write_all

In [ ]:
inl stdin_write_all (stdin : threading.mutex_guard process_child_stdin) (text : string) : () =
    inl stream = text |> sm'.as_bytes
    inl stdin = join stdin
    (!\($'"true; let mut !stdin = !stdin"') : bool) |> ignore
    (!\\(stdin, $'"true; std::io::Write::write_all(&mut *$0, !stream).unwrap()"') : bool) |> ignore

## stdin_flush

In [ ]:
inl stdin_flush (stdin : threading.mutex_guard process_child_stdin) : () =
    inl stdin = join stdin
    (!\($'"true; let mut !stdin = !stdin"') : bool) |> ignore
    (!\\(stdin, $'"true; std::io::Write::flush(&mut *$0).unwrap()"') : bool) |> ignore

## new_process_command

In [ ]:
inl new_process_command (file_name : string) : process_command =
    !\\(file_name, $'"std::process::Command::new(&*$0)"')

## process_stdio_piped

In [ ]:
inl process_stdio_piped () : process_stdio =
    !\($'"std::process::Stdio::piped()"')

## process_command_args

In [ ]:
inl process_command_args (args : am'.vec sm'.std_string) (c : process_command) : rust.ref' (rust.mut' process_command) =
    (!\($'"true; let mut !c = !c"') : bool) |> ignore
    !\\((c, args), $'"std::process::Command::args(&mut $0, &*$1)"')

## process_command_stdout

In [ ]:
inl process_command_stdout (stdio : process_stdio) (c : rust.ref' (rust.mut' process_command)) : rust.ref' (rust.mut' process_command) =
    !\\(c, $'"std::process::Command::stdout($0, std::process::Stdio::piped())"')

## process_command_stderr

In [ ]:
inl process_command_stderr (stdio : process_stdio) (c : rust.ref' (rust.mut' process_command)) : rust.ref' (rust.mut' process_command) =
    !\\(c, $'"std::process::Command::stderr($0, std::process::Stdio::piped())"')

## process_command_stdin

In [ ]:
inl process_command_stdin (stdio : process_stdio) (c : rust.ref' (rust.mut' process_command)) : rust.ref' (rust.mut' process_command) =
    !\\(c, $'"std::process::Command::stdin($0, std::process::Stdio::piped())"')

## process_command_current_dir

In [ ]:
inl process_command_current_dir
    (dir : string)
    (c : rust.ref' (rust.mut' process_command))
    : rust.ref' (rust.mut' process_command)
    =
    !\\(dir, $'"std::process::Command::current_dir(!c, &*$0)"')

## process_command_env

In [ ]:
inl process_command_env
    (key : string)
    (value : string)
    (c : rust.ref' (rust.mut' process_command))
    : rust.ref' (rust.mut' process_command)
    =
    !\\((key, value), $'"std::process::Command::env(!c, &*$0, &*$1)"')

## process_command_spawn

In [ ]:
inl process_command_spawn
    (c : rust.ref' (rust.mut' process_command))
    : resultm.result' process_child stream.io_error
    =
    !\\(c, $'"std::process::Command::spawn($0)"')

## child_wait_with_output

In [ ]:
inl child_wait_with_output
    (child : process_child)
    : resultm.result' process_output stream.io_error
    =
    !\\(child, $'"$0.wait_with_output()"')

## stdio_line

In [ ]:
inl stdio_line
    (stdio : result () ())
    (trace' : bool)
    (channel_sender : threading.arc (threading.mutex (threading.channel_sender sm'.std_string)))
    (line : resultm.result' sm'.std_string stream.io_error)
    : resultm.result' () sm'.std_string
    =
    inl highlight text =
        $'$"\\u001b[4;7m{!text}\\u001b[0m"'
    inl line =
        match
            line
            |> resultm.map_error' sm'.format'
            |> resultm.unbox'
        with
        | Ok line =>
            inl line =
                line
                |> sm'.from_std_string
                // |> sm'.as_bytes
                // |> am'.slice_to_vec
                |> sm'.encoding_encode' (sm'.encoding_utf8' ())
                |> rust.cow_as_ref
                |> sm'.str_from_utf8
                // |> sm'.utf8_decode
                |> resultm.unwrap'
                |> sm'.ref_to_std_string
                // String::from_utf8_lossy(line.as_bytes()).into()
            inl line_log = line |> sm'.from_std_string
            inl text =
                match stdio with
                | Ok () => $'$"> {!line_log}"'
                | Error () => $'$"\! {!line_log}"'
            if trace'
            then trace Verbose (fun () => text) _locals
            else text |> console.write_line
            match stdio with
            | Ok () => line
            | Error () => line |> highlight |> sm'.to_std_string
        | Error e =>
            trace Critical
                fun () => $'$"runtime.stdio_line"'
                fun () => $'$"e: {!e} / {!_locals ()}"'
            e |> highlight |> sm'.to_std_string
    channel_sender
    |> threading.arc_mutex_lock
    |> resultm.unwrap'
    |> threading.mutex_guard_ref
    |> threading.channel_send line
    |> resultm.map_error' sm'.format'

## execute_with_options

In [ ]:
let execute_with_options (options : execution_options) : i32 * string =
    run_target function
        | Fsharp (Native) => fun () =>
            options |> execute_with_options_async |> async.run_synchronously
        | Rust (Native) => fun () =>
            inl command = join options.command
            inl file_name, arguments = command |> split_command
            inl arguments =
                arguments
                |> split_args
                |> am'.to_vec
                |> am'.vec_map sm'.to_std_string

            trace Debug
                fun () => $'$"runtime.execute_with_options"'
                fun () => $'$"file_name: {!file_name} / arguments: %A{!arguments} / options: %A{!options} / {!_locals ()}"'

            fun () =>
                fun () =>
                    file_name
                    |> new_process_command
                    |> process_command_args arguments
                    |> process_command_stdout (process_stdio_piped ())
                    |> process_command_stderr (process_stdio_piped ())
                    |> process_command_stdin (process_stdio_piped ())
                    |> fun command =>
                        match options.working_directory |> optionm'.unbox with
                        | Some working_directory => command |> process_command_current_dir working_directory
                        | None => command
                    |> fun command =>
                        match options.environment_variables with
                        | ;[] => command
                        | vars =>
                            (command, vars |> am'.to_vec)
                            ||> am'.vec_fold' fun command (key, value) =>
                                command |> process_command_env key value
                    |> process_command_spawn
                    |> resultm.map_error' sm'.format'
                    |> resultm.map' (optionm'.some' >> threading.new_arc_mutex)
                    |> resultm.unbox'
                    |> function
                        | Ok child =>
                            // inl stdout =
                            //     child
                            //     |> threading.arc_mutex_lock
                            //     |> resultm.unwrap'
                            //     |> threading.mutex_guard_ref_mut
                            //     |> optionm'.as_mut
                            //     |> optionm'.unwrap
                            //     |> process_child_stdout
                            //     |> optionm'.ref_take
                            //     |> optionm'.unwrap
                            //     |> optionm'.some'
                            //     |> threading.new_arc_mutex
                            // inl stderr =
                            //     child
                            //     |> threading.arc_mutex_lock
                            //     |> resultm.unwrap'
                            //     |> threading.mutex_guard_ref_mut
                            //     |> optionm'.as_mut
                            //     |> optionm'.unwrap
                            //     |> process_child_stderr
                            //     |> optionm'.ref_take
                            //     |> optionm'.unwrap
                            //     |> optionm'.some'
                            //     |> threading.new_arc_mutex
                            // inl stdin =
                            //     child
                            //     |> threading.arc_mutex_lock
                            //     |> resultm.unwrap'
                            //     |> threading.mutex_guard_ref_mut
                            //     |> optionm'.as_mut
                            //     |> optionm'.unwrap
                            //     |> process_child_stdin
                            //     |> optionm'.ref_take
                            //     |> optionm'.unwrap
                            //     |> optionm'.some'
                            //     |> threading.new_arc_mutex

                            inl stdout : process_child_stdout =
                                !\\(child, $'"$0.lock().unwrap().as_mut().unwrap().stdout.take().unwrap()"')
                            inl stderr : process_child_stderr =
                                !\\(child, $'"$0.lock().unwrap().as_mut().unwrap().stderr.take().unwrap()"')
                            inl stdin : process_child_stdin =
                                !\\(child, $'"$0.lock().unwrap().as_mut().unwrap().stdin.take().unwrap()"')
                            inl stdout = stdout |> threading.new_arc_mutex
                            inl stderr = stderr |> threading.new_arc_mutex
                            inl stdin = stdin |> optionm'.some' |> threading.new_arc_mutex
                            inl channel_sender, channel_receiver = threading.new_channel ()
                            inl channel_sender'' = channel_sender |> threading.new_arc_mutex
                            inl channel_sender' = channel_sender |> threading.new_arc_mutex
                            inl channel_receiver' = channel_receiver |> threading.new_arc_mutex

                            inl stdout_handle =
                                fun () =>
                                    stdout
                                    |> threading.arc_mutex_lock
                                    |> resultm.unwrap'
                                    |> threading.mutex_guard_ref_mut
                                    |> stream.decode_reader_bytes_build
                                    |> stream.new_buf_reader
                                    |> stream.buf_read_lines
                                    |> iter.try_for_each fun lines =>
                                        inl channel_sender'' = channel_sender'' |> rust.clone
                                        lines
                                        |> stdio_line (Ok ()) options.trace channel_sender''
                                        |> resultm.to_try
                                |> threading.spawn (1, 0) 1

                            inl stderr_handle =
                                fun () =>
                                    stderr
                                    |> threading.arc_mutex_lock
                                    |> resultm.unwrap'
                                    |> threading.mutex_guard_ref_mut
                                    |> stream.decode_reader_bytes_build
                                    |> stream.new_buf_reader
                                    |> stream.buf_read_lines
                                    |> iter.try_for_each fun lines =>
                                        inl channel_sender' = channel_sender' |> rust.clone
                                        lines
                                        |> stdio_line (Error ()) options.trace channel_sender'
                                        |> resultm.to_try
                                |> threading.spawn (1, 0) 1

                            match options.stdin |> optionm'.unbox with
                            | Some stdin' =>
                                stdin
                                |> threading.arc_mutex_lock
                                |> resultm.unwrap'
                                |> threading.mutex_guard_ref_mut
                                |> optionm'.ref_take
                                |> optionm'.map' threading.new_arc_mutex
                                |> optionm'.unbox
                                |> function
                                    | Some stdin =>
                                        stdin |> stdin'
                                        stdin
                                        |> threading.arc_mutex_lock
                                        |> resultm.unwrap'
                                        |> stdin_flush
                                    | None => ()
                            | None => ()

                            inl output =
                                child
                                |> threading.arc_mutex_lock
                                |> resultm.unwrap'
                                |> threading.mutex_guard_ref_mut
                                |> optionm'.ref_take
                                |> optionm'.unwrap
                                |> child_wait_with_output
                                |> resultm.map_error' sm'.format'

                            [ stdout_handle; stderr_handle ]
                            |> am'.new_vec
                            |> am'.vec_for_each' (threading.join' >> resultm.unwrap' >> resultm.unwrap')

                            match output |> resultm.unbox with
                            | Ok output =>
                                inl exit_code =
                                    output
                                    |> process_output_status
                                    |> process_exit_status_code
                                    |> optionm'.unwrap
                                exit_code, None, Some channel_receiver'
                            | Error error =>
                                trace Critical
                                    fun () => $'$"runtime.execute_with_options / output error"'
                                    fun () => $'$"error: {!error} / {!_locals ()}"'
                                -2i32, error |> Some, None
                        | Error error =>
                            trace Critical
                                fun () => $'$"runtime.execute_with_options / child error"'
                                fun () => $'$"error: {!error} / {!_locals ()}"'
                            -1i32, error |> Some, None
                    |> function
                        | exit_code, std_trace, channel_receiver =>
                            inl std_trace =
                                channel_receiver
                                |> optionm'.box
                                |> optionm'.map' fun channel_receiver =>
                                    channel_receiver
                                    |> threading.arc_mutex_lock
                                    |> resultm.unwrap'
                                    |> iter.iter
                                    |> iter_prototypes.iter_collect''
                                    |> am'.vec_map sm'.from_std_string
                                    |> am'.from_vec
                                    |> fun x => x : _ i32 _
                                    |> seq.of_array
                                    |> sm'.concat "\n"
                                |> optionm'.default_value' (
                                    std_trace
                                    |> optionm.map sm'.from_std_string
                                    |> optionm'.default_value ""
                                )
                            trace Verbose
                                fun () => $'$"runtime.execute_with_options / result"'
                                fun () => $'$"exit_code: {!exit_code} / std_trace.Length: {!std_trace.Length} / {!_locals ()}"'
                            new_pair exit_code std_trace
                |> capture
            |> async.future_init (3, 2) 1
            |> async.block_on
            |> from_pair
        | _ => fun () => null ()

### execute

In [ ]:
inl execute command =
    execution_options fun x => { x with
        command = command
    }
    |> execute_with_options

### test 1

In [ ]:
//// test
///! rust -d chrono encoding_rs encoding_rs_io futures futures-lite regex

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
inl content = "╭─[ 你好，世界！こんにちは世界！ ]─╮"
// inl content = "test"
fun () =>
    inl file_name = join "test.txt"
    inl path = temp_folder </> file_name |> file_system.normalize_path
    inl exit_code, result =
        execute $'\@$"pwsh -c ""[IO.File]::ReadAllText(\'{!path}\')"""'
    exit_code |> _assert_eq 1
    result |> _assert_string_contains "not find file"

    content |> file_system.write_all_text path

    execution_options fun x => { x with
        command = $'\@$"cat ""{!file_name}"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options
    |> ignore

    execution_options fun x => { x with
        command = $'\@$"pwsh -c ""[System.Console]::OutputEncoding = [System.Text.Encoding]::UTF8; [IO.File]::ReadAllText(\'{!file_name}\')"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options
|> fun fn => fn () |> Some
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq 0i32
        output |> _assert_eq content
        true
    | _ => false
|> _assert_eq true
disposable |> use |> ignore

00:00:00 verbose #1 file_system.create_directory / dir: /tmp/!spiral_builder_d1dcde62ab9238eb4feb7d7163f5f76a3bd0cf7be4b24c45691835e49198feb6/20240520-1342-5897-0313-00000006485a
00:00:00   debug #2 runtime.execute_with_options / file_name: pwsh / arguments: ["-c", "[IO.File]::ReadAllText('/tmp/!spiral_builder_d1dcde62ab9238eb4feb7d7163f5f76a3bd0cf7be4b24c45691835e49198feb6/20240520-1342-5897-0313-00000006485a/test.txt')"] / options: (None, "pwsh -c "[IO.File]::ReadAllText('/tmp/!spiral_builder_d1dcde62ab9238eb4feb7d7163f5f76a3bd0cf7be4b24c45691835e49198feb6/20240520-1342-5897-0313-00000006485a/test.txt')"", Array(MutCell([])), None, None, true, None)
00:00:00 verbose #3 ! MethodInvocationException: Exception calling "ReadAllText" with "1" argument(s): "Could not find file '/tmp/!spiral_builder_d1dcde62ab9238eb4feb7d7163f5f76a3bd0cf7be4b24c45691835e49198feb6/20240520-1342-5897-0313-00000006485a/test.txt'."
00:00:00 verbose #4 runtime.execute_with_options / result / exit_code: 1 / std_t

## command

In [ ]:
nominal command = $'clap_Command'

## new_command

In [ ]:
inl new_command (s : rust.static_ref' sm'.str) : command =
    !\\(s, $'"clap::Command::new($0)"')

In [ ]:
//// test
///! rust -d clap

types ()
##"command"
|> new_command
|> sm'.format_pretty'
|> sm'.from_std_string
|> _assert_string_contains "command"

assert_string_contains / actual: "command" / expected: "Command {
    name: "command",
    long_flag: None,
    short_flag: None,
    display_name: None,
    bin_name: None,
    author: None,
    version: None,
    long_version: None,
    about: None,
    long_about: None,
    before_help: None,
    before_long_help: None,
    after_help: None,
    after_long_help: None,
    aliases: [],
    short_flag_aliases: [],
    long_flag_aliases: [],
    usage_str: None,
    usage_name: None,
    help_str: None,
    disp_ord: None,
    template: None,
    settings: AppFlags(
        0,
    ),
    g_settings: AppFlags(
        0,
    ),
    args: MKeyMap {
        args: [],
        keys: [],
    },
    subcommands: [],
    groups: [],
    current_help_heading: None,
    current_disp_ord: Some(
        0,
    ),
    subcommand_value_name: None,
    subcommand_heading: None,
    external_value_parser: None,
    long_help_exists: false,
    deferred: None,
    app_ext: Extensions {
        extensio

## arg

In [ ]:
nominal arg = $'clap_Arg'

## new_arg

In [ ]:
inl new_arg (s : rust.static_ref' sm'.str) : arg =
    !\\(s, $'"clap::Arg::new($0)"')

In [ ]:
//// test
///! rust -d clap

types ()
##"arg"
|> new_arg
|> sm'.format_pretty'
|> sm'.from_std_string
|> _assert_string_contains "arg"

assert_string_contains / actual: "arg" / expected: "Arg {
    id: "arg",
    help: None,
    long_help: None,
    action: None,
    value_parser: None,
    blacklist: [],
    settings: ArgFlags(
        0,
    ),
    overrides: [],
    groups: [],
    requires: [],
    r_ifs: [],
    r_unless: [],
    short: None,
    long: None,
    aliases: [],
    short_aliases: [],
    disp_ord: None,
    val_names: [],
    num_vals: None,
    val_delim: None,
    default_vals: [],
    default_vals_ifs: [],
    terminator: None,
    index: None,
    help_heading: None,
    value_hint: None,
    default_missing_vals: [],
}"


## command_arg

In [ ]:
inl command_arg (arg : arg) (command : command) : command =
    !\\((command, arg), $'"clap::Command::arg($0, $1)"')

## arg_required

In [ ]:
inl arg_required (value : bool) (arg : arg) : arg =
    !\\((arg, value), $'"$0.required($1)"')

## arg_short

In [ ]:
inl arg_short (value : char) (arg : arg) : arg =
    !\\((arg, value), $'"$0.short($1)"')

## arg_long

In [ ]:
inl arg_long (value : rust.static_ref' sm'.str) (arg : arg) : arg =
    !\\((arg, value), $'"$0.long($1)"')

## arg_value_names

In [ ]:
inl arg_value_names (values : array_base (rust.static_ref' sm'.str)) (arg : arg) : arg =
    inl values = values |> am'.to_vec
    !\\((arg, values), $'"$0.value_names($1)"')

## arg_num_args

In [ ]:
inl arg_num_args (value : i32) (arg : arg) : arg =
    !\\((arg, value), $'"$0.num_args($1)"')

## value_range

In [ ]:
nominal value_range = $'clap_builder_ValueRange'

## new_value_range

In [ ]:
inl new_value_range start end : value_range =
    inl len = 0i32 |> unativeint
    inl start, end =
        open am'
        match start, end with
        | Start start, End fn =>
            start, len |> fn
        | End start_fn, End end_fn =>
            start_fn len, end_fn len
    match start, end with
    | start, end when end =. len => !\($'"clap::builder::ValueRange::new(!start..)"')
    | start, end => !\($'"clap::builder::ValueRange::new(!start..!end)"')

## arg_num_args_range

In [ ]:
inl arg_num_args_range (value : value_range) (arg : arg) : arg =
    !\\((arg, value), $'"$0.num_args($1)"')

## arg_value_name

In [ ]:
inl arg_value_name (value : string) (arg : arg) : arg =
    inl value = value |> sm'.as_str
    !\\((arg, value), $'"$0.value_name($1)"')

## value_parser

In [ ]:
nominal value_parser = $'clap_builder_ValueParser'

## possible_value

In [ ]:
nominal possible_value = $'clap_builder_PossibleValue'

## new_possible_value

In [ ]:
inl new_possible_value forall t. (x : t) : possible_value =
    !\\(x, $'"clap::builder::PossibleValue::new(&**$0)"')

## value_parser_possible_values

In [ ]:
inl value_parser_possible_values (values : array_base string) : value_parser =
    inl values =
        values
        |> am'.to_vec
        |> am'.vec_map (sm'.to_std_string >> rust.new_box >> rust.box_leak >> new_possible_value)
    !\\(values, $'"clap::builder::PossibleValuesParser::new($0).into()"')

## value_parser_path_buf

In [ ]:
inl value_parser_path_buf () : value_parser =
    !\($'"clap::value_parser\!(std::path::PathBuf)"')

## value_parser_expr

In [ ]:
inl value_parser_expr (expr : string) : value_parser =
    !\($'"clap::value_parser\!(" + !expr + ").into()"')

## arg_value_parser

In [ ]:
inl arg_value_parser (values : value_parser) (arg : arg) : arg =
    !\\((arg, values), $'"$0.value_parser($1)"')

## arg_union

In [ ]:
inl arg_union forall union_type {to_string}. (fn : union_type -> ()) (arg : arg) : arg =
    arg
    |> arg_value_parser (
        real get_union_fields_untag `union_type ()
        |> fun x => x : list union_type
        |> listm.map to_string
        |> listm'.box
        |> listm'.to_array'
        |> fun (a x : _ i32 _) => x
        |> value_parser_possible_values
    )

## arg_action

In [ ]:
nominal arg_action' = $'clap_ArgAction'

union arg_action =
    | Set
    | Append
    | SetTrue
    | SetFalse
    | Count
    | Help
    | HelpShort
    | HelpLong
    | Version

inl arg_action = function
    | Set => !\($'"clap::ArgAction::Set"') : arg_action'
    | Append => !\($'"clap::ArgAction::Append"') : arg_action'
    | SetTrue => !\($'"clap::ArgAction::SetTrue"') : arg_action'
    | SetFalse => !\($'"clap::ArgAction::SetFalse"') : arg_action'
    | Count => !\($'"clap::ArgAction::Count"') : arg_action'
    | Help => !\($'"clap::ArgAction::Help"') : arg_action'
    | HelpShort => !\($'"clap::ArgAction::HelpShort"') : arg_action'
    | HelpLong => !\($'"clap::ArgAction::HelpLong"') : arg_action'
    | Version => !\($'"clap::ArgAction::Version"') : arg_action'

inl arg_action (value : arg_action) (arg : arg) : arg =
    inl value = value |> arg_action
    !\\((arg, value), $'"$0.action($1)"')

## arg_index

In [ ]:
inl arg_index (value : i32) (arg : arg) : arg =
    !\\((arg, value), $'"$0.index($1)"')

## arg_matches

In [ ]:
nominal arg_matches = $'clap_ArgMatches'

## command_get_matches

In [ ]:
inl command_get_matches (command : command) : arg_matches =
    !\\(command, $'"clap::Command::get_matches($0)"')

## command_get_matches_from

In [ ]:
inl command_get_matches_from (args : array_base string) (command : command) : arg_matches =
    inl args = args |> am'.to_vec |> am'.vec_map sm'.to_std_string
    !\\(command, $'"clap::Command::get_matches_from($0, !args)"')

## command_init_arg

In [ ]:
inl command_init_arg (long, short) fn command =
    command
    |> command_arg (
        new_arg ##long
        |> arg_short short
        |> arg_long ##long
        |> fn
    )

## matches_get_one

In [ ]:
inl matches_get_one forall t. (x : string) (matches : arg_matches) : optionm'.option' t =
    inl x = join x
    inl x = x |> sm'.as_str
    !\\(matches, $'"clap::ArgMatches::get_one(&$0, !x).cloned()"')

## matches_get_flag

In [ ]:
inl matches_get_flag (x : string) (matches : arg_matches) : bool =
    inl x = join x
    inl x = x |> sm'.as_str
    !\($'"clap::ArgMatches::get_flag(&!matches, !x)"')

## matches_get_many

In [ ]:
inl matches_get_many forall t. (x : string) (matches : arg_matches) : optionm'.option' (am'.vec t) =
    inl x = join x
    inl x = x |> sm'.as_str
    !\\(matches, $'"clap::ArgMatches::get_many(&$0, !x).map(|x| x.cloned().into_iter().collect())"')

## matches_get_occurrences

In [ ]:
inl matches_get_occurrences (x : string) (matches : arg_matches) : optionm'.option' (array_base sm'.std_string) =
    inl x = join x
    inl x = x |> sm'.as_str
    !\($'"clap::ArgMatches::get_occurrences(&!matches, !x).cloned()"')

## matches_subcommand

In [ ]:
inl matches_subcommand (matches : arg_matches) : optionm'.option' (sm'.std_string * arg_matches) =
    !\\((matches, sm'.ref_to_std_string), $'"clap::ArgMatches::subcommand(Box::leak(Box::new($0))).map(|(a, b)| ($1(a), b.clone()))"')

## matches_values_of

In [ ]:
inl matches_values_of (x : string) (matches : arg_matches) : array_base sm'.std_string =
    !\\((matches, x), $'"clap::ArgMatches::values_of($0, &*$1)"')

## command_debug_assert

In [ ]:
inl command_debug_assert (command : command) : () =
    !\\(command, $'"clap::Command::debug_assert($0)"')

## command_subcommand_required

In [ ]:
inl command_subcommand_required (value : bool) (command : command) : command =
    !\\(command, $'"clap::Command::subcommand_required($0, !value)"')

## command_subcommand

In [ ]:
inl command_subcommand (subcommand : command) (command : command) : command =
    !\\(command, $'"clap::Command::subcommand($0, !subcommand)"')

## main

In [ ]:
inl main () =
    types ()
    init_trace_state None
    $"let is_windows () = !is_windows ()" : ()
    $"let get_executable_suffix () = !get_executable_suffix ()" : ()
    $"let execute_async x = !execute_async x" : ()
    $"let execute_with_options_async x = !execute_with_options_async x" : ()
    inl execution_options fn =
        execution_options fun x =>
            x
            |> heap
            |> fn
            |> fun x => !x
    $"let execution_options x = !execution_options x" : ()
    $"let split_args x = !split_args x" : ()